In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vsopy import util, data
import astropy.units as u
from SetupSession import use_dark_theme


In [ ]:
import vsopy.util
import vsopy.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='2025/20250126'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vsopy.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vsopy.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vsopy.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir / 'V'
sd = vsopy.data.StarData(layout.charts_dir)

In [ ]:
import ccdproc as ccdp
import vsopy.reduce

ifc = ccdp.ImageFileCollection(IMAGE_DIR)
path = IMAGE_DIR / ifc.summary['file'][0]
image  = vsopy.reduce.load_and_solve(path, session_layout.solved_dir)
matcher = vsopy.reduce.CalibrationMatcher(layout.calibr_dir, temp_tolerance=2*u.K)
cal = matcher.match(image.header)
reduced = vsopy.reduce.calibrate_image(image,
                        dark=cal.dark,
                        flat=cal.flat)


In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.coordinates import SkyCoord
import vsopy.phot
from astropy.table import QTable
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import AsinhStretch, ImageNormalize, MinMaxInterval
from matplotlib.patches import Circle, RegularPolygon, Ellipse


In [ ]:
detected = QTable.read(session_layout.root_dir / 'detected_stars.ecsv')
np.max(detected['count'])
detected

In [ ]:
%matplotlib ipympl
from astropy.visualization import AsinhStretch, LogStretch, ImageNormalize, MinMaxInterval
from matplotlib import cm

cmap = cm.get_cmap('gray').reversed()

alpha = 0.001
interval = MinMaxInterval()
fig = plt.figure(figsize=(16.00, 16.00))
ax = plt.subplot(projection=reduced.wcs)
mean, median, std = sigma_clipped_stats(reduced.data, sigma=3.0)
vmin, vmax = interval.get_limits(reduced.data)
norm = ImageNormalize(vmin=mean+2*std, vmax=vmax, stretch=LogStretch(10000))
ax.imshow(reduced.data, origin='lower', norm=norm, cmap=cmap)
for star in detected:
    r = 10 #2*(20 - t['M']['mag'].value)
    c = reduced.wcs.world_to_pixel(star['radec2000'])
    ax.add_patch(Circle(c, r, color='red', alpha=.5, fill=False))
# for s in cat_stars:
#     r = 2*(20 - s['Vmag'])
#     c = reduced.wcs.world_to_pixel(s['radec2000'])
#     ax.add_patch(Circle(c, r, color='blue', alpha=.5))
# for s in vars:
#     r = 40
#     c = reduced.wcs.world_to_pixel(s['radec2000'])
#     ax.add_patch(RegularPolygon(c, 4, radius=r, color='yellow', fill=False, alpha=.5))
# for s in cat_gals:
#     r = 2*(20 - s['Vmag'])
#     c = reduced.wcs.world_to_pixel(s['radec2000'])
#     ax.add_patch(Circle(c, r, color='yellow', alpha=.5))
plt.show()
